## Fake News Detection

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
path = "./Desktop/ProjectGurukul/Fake News Detection/"
true_df = pd.read_csv(path + 'True.csv')
fake_df = pd.read_csv(path + 'Fake.csv')

In [ ]:
true_df['label'] = 0

In [ ]:
fake_df['label'] = 1

In [ ]:
true_df.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


In [ ]:
fake_df.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [ ]:
true_df = true_df[['text','label']]
fake_df = fake_df[['text','label']]

In [ ]:
dataset = pd.concat([true_df , fake_df])

In [ ]:
dataset.shape

(44898, 2)

### Null values

In [ ]:
dataset.isnull().sum() # no null values

text     0
label    0
dtype: int64

### Balanced or Unbalanced dataset

In [ ]:
dataset['label'].value_counts()

1    23481
0    21417
Name: label, dtype: int64

In [ ]:
true_df.shape # true news

(21417, 2)

In [ ]:
fake_df.shape # fake news

(23481, 2)

### Shuffle or Resample

In [ ]:
dataset = dataset.sample(frac = 1)

In [ ]:
dataset.head(20)

,text,label
22161,21st Century Wire says It was an awkward fit o...,1
17723,MONROVIA (Reuters) - One of Liberia s leading ...,0
13759,MOSCOW (Reuters) - Former Russian economy mini...,0
9387,(Reuters) - Officials from 11 U.S. states sued...,0
18704,COLBERT LANGUAGE WARNING! This late night ho...,1
2476,"Kellyanne Conway, who serves as a senior advis...",1
16571,"I ll bet you re thinking this is a joke, right...",1
6836,WASHINGTON (Reuters) - U.S. President-elect Do...,0
13742,Don t buy into the media lie that every LEGAL ...,1
468,"Donald Trump has the maturity of a toddler, an...",1


In [ ]:
import nltk

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
stopwords = stopwords.words('english')

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
def clean_data(text):
    text = text.lower() 
    text = re.sub('[^a-zA-Z]' , ' ' , text)
    token = row.split() 
    token = [lemmatizer.lemmatize(word) for word in token if not word in stopwords]  
    clean_news = ' '.join(news) 
    
    return clean_news 

In [ ]:
dataset['text'] = dataset['text'].apply(lambda x : clean_data(x))

In [ ]:
dataset.isnull().sum()

text     0
label    0
dtype: int64

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer(max_features = 50000 , lowercase=False , ngram_range=(1,2))

In [ ]:
X = dataset.iloc[:35000,0]
y = dataset.iloc[:35000,1]

In [ ]:
X.head()

22161    st century wire say awkward fit nerve donald m...
17723    monrovia reuters one liberia leading political...
13759    moscow reuters former russian economy minister...
9387     reuters official u state sued obama administra...
18704    colbert language warning late night host one a...
Name: text, dtype: object

In [ ]:
y.head()

22161    1
17723    0
13759    0
9387     0
18704    1
Name: label, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
train_X , test_X , train_y , test_y = train_test_split(X , y , test_size = 0.2 ,random_state = 0)

In [ ]:
vec_train = vectorizer.fit_transform(train_data)

In [ ]:
vec_train = vec_train.toarray()

In [ ]:
vec_test = vectorizer.transform(test_X).toarray()

In [ ]:
train_data = pd.DataFrame(vec_train , columns=vectorizer.get_feature_names())
test_data = pd.DataFrame(vec_test , columns= vectorizer.get_feature_names())

## Multinomial NB

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
from sklearn.metrics import accuracy_score,classification_report

In [ ]:
clf = MultinomialNB()

In [ ]:
clf.fit(train_data, train_y)
predictions  = clf.predict(test_data)

In [ ]:
print(classification_report(test_y , predictions))

              precision    recall  f1-score   support

           0       0.96      0.94      0.95      3326
           1       0.95      0.96      0.96      3674

    accuracy                           0.95      7000
   macro avg       0.95      0.95      0.95      7000
weighted avg       0.95      0.95      0.95      7000



Now predict on both train set

In [ ]:
predictions_train = clf.predict(train_data)
print(classification_report(train_y , predictions_train))

              precision    recall  f1-score   support

           0       0.96      0.95      0.96     13385
           1       0.96      0.96      0.96     14615

    accuracy                           0.96     28000
   macro avg       0.96      0.96      0.96     28000
weighted avg       0.96      0.96      0.96     28000



In [ ]:
accuracy_score(train_y , predictions_train)

0.9584642857142858

In [ ]:
accuracy_score(test_y , predictions)

0.9531428571428572